<a href="https://colab.research.google.com/github/costaljorge/Algoritmo_Genetico_Lotomania/blob/main/Algoritmo_Genetico_Lotomania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Um **algoritmo genético** é uma técnica de otimização inspirada na evolução natural. Ele usa mecanismos como herança, mutação, seleção e crossover (reprodução) para gerar soluções para um problema. Inicia-se com uma população de soluções possíveis (indivíduos) que evoluem ao longo das gerações. Em cada geração, a aptidão de cada indivíduo é avaliada, os mais aptos são selecionados para reprodução e geram novos indivíduos. Este processo é repetido até que uma condição de parada seja atingida. Algoritmos genéticos são úteis para problemas de otimização complexos e não-lineares.

In [ ]:
# Instalando a Biblioteca python para Algoritmos Genéticos (deap)
!pip install deap

In [ ]:
# Leitura do aquivo
# O arquivo esta instalado na area de trabalho, bloco de notas no formato .txt
# Cada linha de iniciar com a tag < e ser fechada com a tag >
# Por exemplo <07 08 10 12 19 21 25 27 37 43 45 48 69 75 80 82 84 88 92 93>
# Para esse exemplo formam inserida 5 linhas
# Abrir o arquivo para leitura
with open('/content/Loteria.txt', 'r') as file:
    # Ler as linhas do arquivo
    lines = file.readlines()

# Inicializar a lista de listas
data = []

# Iterar sobre as linhas
for line in lines:
    # Remover os caracteres de tag <>
    line = line.strip('<>').strip('>')
    # Separar os números e converter para inteiros
    numbers = [int(num.strip('>')) for num in line.split()]
    # Adicionar a lista de números à lista de dados
    data.append(numbers)

In [ ]:
import random
from deap import base, creator, tools

# Definição da função de interseção com as linhas
def intersection_size(chromosome, lines):
    total_intersection = 0
    for line in lines:
        intersection = set(chromosome) & set(line)
        total_intersection += min(len(intersection), 20)
    return total_intersection

# Configuração do algoritmo genético
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
# Cromossomo de tamanho 50 com valores únicos de 0 a 99
toolbox.register("attr_int", random.sample, range(100), 50)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_int)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=99, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", intersection_size, lines=data)

# Execução do algoritmo genético
population = toolbox.population(n=100)
num_generations = 50
for gen in range(num_generations):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = (fit,)

    population[:] = offspring

# Encontrar o melhor indivíduo
best_ind = tools.selBest(population, 1)[0]
print("Melhor Interseção:", intersection_size(best_ind,data))

best_ind_sorted = sorted(best_ind)
print("Melhor Indivíduo:", best_ind_sorted)
